# Retriever
- 비정형 질의(query)를 입력 받아 Vector store에서 관련된 내용을 검색하는 기능을 제공하는 인터페이스로 다양한 데이터 소스에서 정보를 검색하여 대규모 언어 모델(LLM) 기반 애플리케이션의 **정확성을** 향상시키는 데 핵심적인 역할을 한다.

![RAG](figures/rag2.png)

## 주요 특징
- **다양한 데이터 소스 지원**
	- Retriever는 벡터 스토어, 그래프 데이터베이스, 관계형 데이터베이스 등 여러 종류의 검색 시스템과 상호작용할 수 있는 통일된 인터페이스를 제공한다다.
- **간단한 인터페이스**: Retriever는 문자열 형태의 쿼리를 입력받아 관련 문서의 리스트를 반환한다. 이러한 단순한 구조 덕분에 다양한 검색 시스템과 쉽게 통합할 수 있다. 


## 다양한 Retriever 방식

**Retriever**란, 사용자의 질문(쿼리)에 가장 관련성 높은 정보를 찾아주는 구성 요소이다. 주로 검색 기반 질문응답 시스템(RAG, Retrieval-Augmented Generation)에서 사용된다. 다음은 자주 사용되는 다양한 Retriever의 유형과 그 특징이다.

1. **벡터 스토어(Vector Store) Retriever**
   - VectorStore로 부터 유사도를 기반으로 검색하는 가장 기본 Retriever
   - 텍스트 조각(청크)마다 **임베딩(embedding)을** 생성하여 벡터 공간에 저장하고, 쿼리 임베딩과의 **코사인 유사도(cosine similarity)** 등을 기반으로 유사한 텍스트를 검색한다.
   - 검색 속도가 빠르고 구현이 간단하여, 기본적인 검색 시스템을 구축할 때 적합하다.
2. **[ParentDocumentRetriever](https://python.langchain.com/docs/how_to/parent_document_retriever/)**
   - 하나의 문서를 여러 청크로 나눈 뒤 각각을 인덱싱하고, 쿼리와 가장 유사한 청크를 찾은 다음 해당 청크가 속한 **전체 원본 문서**를 반환한다.
   - 작은 정보 조각들이 모여 하나의 문서를 구성할 때 유용하며, 문맥을 넓게 유지할 수 있다.
3. **[MultiVectorRetriever](https://python.langchain.com/docs/how_to/multi_vector/)**
   - 각 문서에 대해 요약을 하거나, 가상의 질문을 생성하거나, 사람이 중요한 내용을 직접 추가하여 문서당 여러 개의 임베딩 벡터를 생성한다.
   - 텍스트 전체보다 더 핵심적인 정보가 검색에 반영되도록 하고자 할 때 효과적이다.
   - 특히, 문서가 길거나, 중요한 내용이 문서의 특정 부분에 집중되어 있는 경우에 유리하다.
4. **[SelfQueryRetriever](https://python.langchain.com/docs/how_to/self_query/)**
   - 대규모 언어 모델(LLM, Large Language Model)을 활용하여 사용자의 질문을 적절한 검색어와 **메타데이터(metadata)** 필터로 자동 변환한다.
   - 예를 들어, 문서의 작성자, 날짜, 태그와 같은 메타데이터를 기준으로 검색할 수 있다.
   - 문서 자체의 내용뿐만 아니라, 문서에 부가된 속성 정보에 대해 질문할 때 유용하다.
5. **[ContextualCompressionRetriever](https://python.langchain.com/docs/how_to/contextual_compression/)**
   - 기존 Retriever와 조합되어 사용된다.
   - 먼저 일반적인 검색을 수행한 후, 검색된 문서들에서 쿼리와 관련 없는 불필요한 정보를 제거하고 핵심 내용만을 추출하여 반환한다.
   - 정보를 요약하거나 압축하여 LLM에 전달할 문서 길이를 줄일 때 유용하다.
6. **[MultiQueryRetriever](https://python.langchain.com/docs/how_to/MultiQueryRetriever/)**
   - LLM을 이용해 하나의 쿼리로부터 여러 가지 변형된 쿼리를 생성하고, 각 쿼리에 대해 검색을 수행한 뒤 결과를 합치는 방식.
   - 다양한 표현에 강해 검색 범위를 넓히고 성능을 높인다.
7. **[EnsembleRetriever](https://python.langchain.com/docs/how_to/ensemble_retriever/)**
   - 여러 개의 Retriever(예: BM25, 벡터 기반 등)를 결합해 가중치를 기반으로 결과를 조합(re-ranking)한다.
   - 서로 다른 장점을 가진 Retriever를 하나로 묶어 성능을 강화한다.

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# Load -> chunking -> embedding -> Store
text_path = "data/olympic.txt"
collection_name = "olympic_info"
persist_directory = "vector_store/chroma/olympic_info"

## load + split
loader = TextLoader(text_path, encoding="utf-8")
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

docs = loader.load_and_split(splitter)
print(len(docs))

# VectorStore와 연결
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")
# vector_store = Chroma.from_documents(docs=docs,  # 연결하면서 문서 추가.
vector_store = Chroma(  # 연결
    embedding_function=embedding_model,
    collection_name=collection_name,
    persist_directory=persist_directory
) 

61


In [5]:
# Vector DB에 문서 추가.
add_ids = vector_store.add_documents(docs)

In [6]:
# add_ids
vector_store._collection.count()

61

In [7]:
# VectorStore(Chroma DB와 연결)로부터 검색(Retrieve)하는 Retriever를 생성.
retriever = vector_store.as_retriever()
result_docs = retriever.invoke("올림픽과 관련된 논란들은 무엇이 있나요?") # 검색할 쿼리를 문자열로 전달.

In [8]:
result_docs
# prompt = query + result_docs ===> LLM

[Document(id='11040a56-2cc8-4aed-8899-eaf2f33b1ec2', metadata={'source': 'data/olympic.txt'}, page_content='올림픽'),
 Document(id='d397d7a6-be74-4d2d-b123-7a4f459b2359', metadata={'source': 'data/olympic.txt'}, page_content='현대 올림픽에서는 프로 선수의 참가 불허가 많은 분쟁을 가져왔다. 1912년 하계 올림픽의 근대 5종 경기와 10종 경기에서 우승한 짐 소프는 올림픽에 나가기 전에 준프로야구선수로 활동했다는 게 나중에 밝혀져 메달이 박탈되었다. 소프는 후에 동정적 여론의 힘을 업고 1983년에 메달을 돌려받게 된다. 1936년 동계 올림픽 때 스위스와 오스트리아 스키선수들은 돈을 벌기 위해 스포츠를 했는데 이러한 행동이 아마추어 정신에 위배된다고 결정되어 그들은 스키종목에 참가할 수 없었다.'),
 Document(id='3ebe03b7-75f1-4acc-a21d-bfe61610d159', metadata={'source': 'data/olympic.txt'}, page_content='쿠베르탱의 생각과는 달리, 올림픽이 세계에 완벽한 평화를 가져다주지는 못했다. 실제로 제1차 세계대전으로 인해 독일 베를린에서 열리기로 했던 제6회 1916년 하계 올림픽이 취소되었고, 제2차 세계대전 때는 일본 도쿄에서 열리기로 했던 제12회 1940년 하계 올림픽, 삿포로에서 열리기로 했던 1940년 동계 올림픽, 영국 런던에서 열리기로 했던 제13회 1944년 하계 올림픽, 이탈리아 코르티나담페초에서 열릴 예정인 1944년 동계 올림픽이 취소되었다. 베이징에서 열린 2008년 하계 올림픽 개막식날 조지아와 러시아 간의 2008년 남오세티아 전쟁이 일어나기도 했다. 부시 대통령과 푸틴 대통령이 이 올림픽을 보러 왔으며 중국 주석인 후진타오가 주최한 오찬에 참석해서 이 현안에 대해 논의하기도 했다. 

In [9]:
# 검색 설정들을 넣어서 retriever생성
retriever2 = vector_store.as_retriever(
    search_type="similarity", # "similarity"(기본), "similarity_score_threshold", "mmr"
    search_kwargs={"k":10,  # 검색 메소드(similarity_search_xxxx())의 파라미터를 dictionary로 전달.
                #    "filter":{"source":"abc"}
                   }  
)
result_docs2 = retriever2.invoke("올림픽과 관련된 논란들은 무엇이 있나요?")
result_docs2

[Document(id='11040a56-2cc8-4aed-8899-eaf2f33b1ec2', metadata={'source': 'data/olympic.txt'}, page_content='올림픽'),
 Document(id='d397d7a6-be74-4d2d-b123-7a4f459b2359', metadata={'source': 'data/olympic.txt'}, page_content='현대 올림픽에서는 프로 선수의 참가 불허가 많은 분쟁을 가져왔다. 1912년 하계 올림픽의 근대 5종 경기와 10종 경기에서 우승한 짐 소프는 올림픽에 나가기 전에 준프로야구선수로 활동했다는 게 나중에 밝혀져 메달이 박탈되었다. 소프는 후에 동정적 여론의 힘을 업고 1983년에 메달을 돌려받게 된다. 1936년 동계 올림픽 때 스위스와 오스트리아 스키선수들은 돈을 벌기 위해 스포츠를 했는데 이러한 행동이 아마추어 정신에 위배된다고 결정되어 그들은 스키종목에 참가할 수 없었다.'),
 Document(id='3ebe03b7-75f1-4acc-a21d-bfe61610d159', metadata={'source': 'data/olympic.txt'}, page_content='쿠베르탱의 생각과는 달리, 올림픽이 세계에 완벽한 평화를 가져다주지는 못했다. 실제로 제1차 세계대전으로 인해 독일 베를린에서 열리기로 했던 제6회 1916년 하계 올림픽이 취소되었고, 제2차 세계대전 때는 일본 도쿄에서 열리기로 했던 제12회 1940년 하계 올림픽, 삿포로에서 열리기로 했던 1940년 동계 올림픽, 영국 런던에서 열리기로 했던 제13회 1944년 하계 올림픽, 이탈리아 코르티나담페초에서 열릴 예정인 1944년 동계 올림픽이 취소되었다. 베이징에서 열린 2008년 하계 올림픽 개막식날 조지아와 러시아 간의 2008년 남오세티아 전쟁이 일어나기도 했다. 부시 대통령과 푸틴 대통령이 이 올림픽을 보러 왔으며 중국 주석인 후진타오가 주최한 오찬에 참석해서 이 현안에 대해 논의하기도 했다. 

In [10]:
retriever3 = vector_store.as_retriever(
    search_type="similarity_score_threshold", # "similarity"(기본), "similarity_score_threshold", "mmr"
    search_kwargs={"k":10, "score_threshold":0.3}  # 유사도 점수가 지정한 값 이상인 것만 조회.
)
result_docs3 = retriever3.invoke("올림픽과 관련된 논란들은 무엇이 있나요?")
result_docs3

[Document(id='11040a56-2cc8-4aed-8899-eaf2f33b1ec2', metadata={'source': 'data/olympic.txt'}, page_content='올림픽'),
 Document(id='d397d7a6-be74-4d2d-b123-7a4f459b2359', metadata={'source': 'data/olympic.txt'}, page_content='현대 올림픽에서는 프로 선수의 참가 불허가 많은 분쟁을 가져왔다. 1912년 하계 올림픽의 근대 5종 경기와 10종 경기에서 우승한 짐 소프는 올림픽에 나가기 전에 준프로야구선수로 활동했다는 게 나중에 밝혀져 메달이 박탈되었다. 소프는 후에 동정적 여론의 힘을 업고 1983년에 메달을 돌려받게 된다. 1936년 동계 올림픽 때 스위스와 오스트리아 스키선수들은 돈을 벌기 위해 스포츠를 했는데 이러한 행동이 아마추어 정신에 위배된다고 결정되어 그들은 스키종목에 참가할 수 없었다.')]

In [11]:
retriever4 = vector_store.as_retriever(
    search_type="mmr", # "similarity"(기본), "similarity_score_threshold", "mmr"
    search_kwargs={"k":5, "fetch_k":20, "lambda_mult":0.5} # mmr 파라미터.
)
result_docs4 = retriever4.invoke("올림픽과 관련된 논란들은 무엇이 있나요?")
result_docs4

[Document(id='11040a56-2cc8-4aed-8899-eaf2f33b1ec2', metadata={'source': 'data/olympic.txt'}, page_content='올림픽'),
 Document(id='d397d7a6-be74-4d2d-b123-7a4f459b2359', metadata={'source': 'data/olympic.txt'}, page_content='현대 올림픽에서는 프로 선수의 참가 불허가 많은 분쟁을 가져왔다. 1912년 하계 올림픽의 근대 5종 경기와 10종 경기에서 우승한 짐 소프는 올림픽에 나가기 전에 준프로야구선수로 활동했다는 게 나중에 밝혀져 메달이 박탈되었다. 소프는 후에 동정적 여론의 힘을 업고 1983년에 메달을 돌려받게 된다. 1936년 동계 올림픽 때 스위스와 오스트리아 스키선수들은 돈을 벌기 위해 스포츠를 했는데 이러한 행동이 아마추어 정신에 위배된다고 결정되어 그들은 스키종목에 참가할 수 없었다.'),
 Document(id='9d3fbc59-7caf-4f45-a709-4192ec3ab2a8', metadata={'source': 'data/olympic.txt'}, page_content='국제 올림픽 위원회(이하 IOC로 지칭)는 몇몇 위원들이 한 행위에 대해서 비판을 받고 있다. 그 예로 IOC 위원장이었던 에이버리 브런디지와 후안 안토니오 사마란치가 대표적인 사람이다. 브런디지는 20년 넘게 IOC 위원장직을 맡았고 임기 중에 올림픽을 정치적으로 휘말려들지 않게 하기 위해 보호했다. 그러나 그는 남아프리카 공화국 대표단에게 아파르트헤이트와 관련된 이슈를 건드리고 반유대정책을 함으로써 비난을 받았다. 사마란치 위원장 시기 때는 족벌 정치와 부패로 비난받았다. 사마란치가 스페인에서 프랑코 정권에 협력했다는 것도 비판의 이유가 되었다.'),
 Document(id='9d8050fb-7d69-439d-9fc5-493b626e9395', metadata={'source': 'data/olym

In [12]:
# retriever | 질문과 검색결과를 프롬프트로 생성 | LLM | output => 체인
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

In [13]:
# RAG 프롬프트 템플릿 -> 답변을 context를 기반으로 해야된다. 라는 표현이 들어가야 함.
template = """# Instruction:
당신은 정확한 정보 제공을 우선시하는 인공지능 어시스턴트입니다.
주어진 Context에 포함된 정보만 사용해서 질문에 답변하세요.
Context에 질문에 대한 명확한 정보가 있는 경우 그 내용을 바탕으로 답변하세요.
Context에 질문에 대한 명확한 정보없을 경우 "정보가 부족해서 답을 알 수 없습니다." 라고 대답합니다.
절대 Context에 없는 내용을 추측하거나 일반 상식을 이용해 답을 만들어서 대답하지 않습니다.

# Context:
{context}

# 질문:
{query}
"""
prompt_template = PromptTemplate(template=template)

In [14]:
from langchain_core.documents import Document
def format_docs(docs:list[Document]) -> str:
    """
    Retriever가 검색한 문서들에서 page_content(문서 내용) 만 추출해서 반환.
    추출된 문서들의 내용을 "\n\n"로 연결한다.
    Args:
        docs(list[Document]) - 검색한 문서 리스트
    Returns:
        str - 문서1내용+\n\n문서2내용+\n\n .. 
    """
    return "\n\n".join(doc.page_content for doc in docs)

In [15]:
# print(format_docs(result_docs))

In [16]:
# @chain
# def final_chain(query:str):
#     docs = retriever.invoke(query)
#     docs = format_docs(docs)
#     prompt = prompt_template.invoke({"context":docs, "query":query})
#     result = model.invoke(prompt)
#     return StrOutputParser().invoke(result)

# final_chain.invoke("올림픽 설명해줘.")

In [17]:
# chain 구성: {retriever, 질문} -> prompt_template -> llm -> output parser
model = ChatOpenAI(model_name="gpt-4.1-mini")
chain = ({"context":retriever|format_docs, "query":RunnablePassthrough()} 
         | prompt_template
         | model
         | StrOutputParser()
        )

In [18]:
# 질의
query = "올림픽과 관련된 논란에 대해 알려줘."
result = chain.invoke(query)

In [19]:
print(result)

올림픽과 관련된 논란으로는 다음과 같은 사례들이 있습니다.

1. 1998년에 몇몇 국제 올림픽 위원회(IOC) 위원들이 2002년 솔트레이크 시티 동계 올림픽 유치 과정에서 미국 측으로부터 뇌물청탁을 받았다는 것이 폭로되었습니다. 이로 인해 IOC는 해당 위원 중 4명이 자진 사퇴하고 6명이 강제 퇴출되도록 조사를 진행했습니다. 이 사건은 이후 개최지 선정 과정에서의 부정행위를 방지하기 위한 IOC 개혁의 계기가 되었습니다.

2. 2004년 8월에 BBC 다큐멘터리 '파노라마'에서는 '매수된 올림픽'이라는 주제로 2012년 하계 올림픽 개최지 선정과 관련된 뇌물 의혹을 다루었습니다. 이 방송에서는 특정 후보 도시가 IOC 위원들에게 뇌물을 제공했을 가능성을 제기했고, 프랑스 파리 시장 베르트랑 들라노에는 당시 영국 총리 토니 블레어와 런던 올림픽 유치위원회가 입후보 규정을 위반했다고 비난했습니다. 그러나 이 주장에 대한 체계적인 조사는 이루어지지 않았습니다.

3. 2006년 동계 올림픽을 유치한 토리노 또한 유치 과정에서 논란에 휩싸였으며, 스위스 국적 IOC 위원인 마크 호들러가 이 논쟁의 중심 인물이 되기도 했습니다.

이와 같이 올림픽 개최지 선정 과정에서 뇌물과 관련된 여러 논란이 발생한 바 있습니다.


In [20]:
chain.invoke("구름의 종류에 대해서 설명해줘.")

'정보가 부족해서 답을 알 수 없습니다.'

# TODO 다음을 작성한다.

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough


collection_name = "olympic_docs"
persist_directory = "vector_store/chroma/olympic"

# Text Loading


# Split


# Vector Store 생성


# Retriever 생성 - "mmr" 방식



In [ ]:
# Prompt Template 생성



# Chain 구성




In [ ]:
# Chain을 이용해 질의


